In [1]:
import pandas as pd
import glob

In [2]:
ct = 'DG_Glut'
degs = pd.read_csv(f"New.Domain.Genes.Expression/{ct}/DiffExpress.DomainGene.csv", index_col = 0)
degs =degs[(degs['log2(18mo/2mo)'] <-0.2) & (degs['fdr'] < 1e-3)]

In [3]:
# degs = []
# for file_path in glob.glob(f"New.Domain.Genes.Expression/*/DiffExpress.DomainGene.csv"):
#     ct = file_path.split('/')[-2]
#     tmpdf = pd.read_csv(file_path, index_col =0)
#     tmpdf['celltype'] = ct
#     DEGs.append(tmpdf)

# degs = pd.concat(degs)
# degs =degs[(degs['log2(18mo/2mo)'] <-0.2) & (degs['fdr'] < 1e-3)]

In [4]:
cemba_corr = pd.read_csv('/data/CembaData/TotalGeneDMRTF.NNZCorrRecords.All.Filtered.csv', index_col =0)

In [5]:
shared_genes = degs.index.intersection(cemba_corr['gene_name'])
len(shared_genes)

129

In [6]:
degs = degs.loc[shared_genes]
deg_cember_corr = cemba_corr[cemba_corr['gene_name'].isin(shared_genes)]

## find examples

In [7]:
# find regions outside the splitted domain
domain_info = pd.read_csv(f'New.Domain.Genes.Expression/{ct}/All.Overlap.Domain.Genes.csv', index_col =0).set_index('gene_name')
domain_info = domain_info[['chrom','gene_start','gene_end','tad_start','tad_end','start','end']]
domain_info.columns = ['chrom','gene_start','gene_end','tad_start','tad_end','boundary_start','boundary_end']

deg_domain_info = domain_info.loc[shared_genes].drop_duplicates(keep = 'first')

In [12]:
# # genes with diff bounday not as boundary in young
# all_diff = pd.read_csv('ALL.Diff.Domain.csv', index_col =0)
# all_diff = all_diff[all_diff['celltype'] == ct]
# all_diff = all_diff[(all_diff['8wk'] < 0.15) ] #& (all_diff['18mo-8wk'] > 0.05)

# deg_domain_info['domain_id'] = deg_domain_info['chrom'] + '_' + (deg_domain_info['tad_start'] // 25000).astype(str)
# deg_domain_info = deg_domain_info[deg_domain_info['domain_id'].isin(all_diff.index)]

# deg_cember_corr = deg_cember_corr[deg_cember_corr['gene_name'].isin(deg_domain_info.index)]

In [8]:
gene_enhancer_region = {}

for _,row in deg_domain_info.iterrows():
    if row['gene_start'] < row['boundary_start']:
        enhancer_region  = (row['tad_start'], row['boundary_end'])
    elif row['gene_end'] > row['boundary_end']:
        enhancer_region  = (row['boundary_start'], row['tad_end'])
    gene_enhancer_region[row.name] = enhancer_region

In [9]:
from collections import defaultdict
gene_dmrs = defaultdict(set)

for gene , tmpdf in deg_cember_corr.groupby('gene_name'):
    enhancer_region = gene_enhancer_region[gene]
    for _dmr in tmpdf['dmr']:
        dmr_pos  = int(_dmr.split('-')[-1])
        if dmr_pos < enhancer_region[0] or  dmr_pos > enhancer_region[1]:
            gene_dmrs[gene].add(_dmr)

In [10]:
tmp = deg_cember_corr[deg_cember_corr['gene_name'].isin(gene_dmrs.keys())]

## find example

In [11]:
gene_name = 'Pigk'

In [12]:
promoter = pd.read_csv('/data/ref/mouse/genome_anno/mm10_promoter.bed', sep = '\t', header = None)
promoter[promoter[5] == gene_name]

,0,1,2,3,4,5,6
24609,chr3,152711599,152716600,ENSMUSG00000039047.13,+,Pigk,protein_coding


In [13]:
tmp = deg_cember_corr[deg_cember_corr['gene_name'] == gene_name]
tmp[tmp['dmr'].isin(gene_dmrs[gene_name])].sort_values('gene-dmr-corr', ascending = False).head(5)

,dmr,tf,loop,tf-self-loop,gene-dmr-corr,gene-dot-corr,tf-dmr-corr,gene-tf-corr,final-corr,gene_name,tf_name
gene,,,,,,,,,,,
ENSMUSG00000039047,chr3-152978527,ENSMUSG00000042903,chr3-15280-15297,False,0.852051,0.505525,0.356201,0.3110,0.467375,Pigk,Foxo4
ENSMUSG00000039047,chr3-152974381,ENSMUSG00000033863,chr3-15280-15297,False,0.815918,0.505525,0.472412,0.3303,0.503722,Pigk,Klf9
ENSMUSG00000039047,chr3-152974381,ENSMUSG00000034041,chr3-15280-15297,False,0.815918,0.505525,0.408936,0.3335,0.487062,Pigk,Lyl1
ENSMUSG00000039047,chr3-152974381,ENSMUSG00000023034,chr3-15280-15297,False,0.815918,0.505525,0.513184,0.3594,0.525167,Pigk,Nr4a1
ENSMUSG00000039047,chr3-152974381,ENSMUSG00000009471,chr3-15280-15297,False,0.815918,0.505525,0.554199,0.3470,0.530675,Pigk,Myod1


## test if less contact

In [20]:
import cooler

In [18]:
leg = [f"{ct}.{age}" for age in ['8wk','9mo','18mo']]

In [14]:
res = 10000
promoter = 'chr3-152711599-152716600'

In [22]:
res = 10000 
_loop = 'chr3-15280-15297'
chrom, loop_start, loop_end = _loop.split('-')
loop_start = int(loop_start) * res
loop_end = int(loop_end) * res
start = loop_start - res
end = loop_end + res

In [26]:
age_count = {}
for group in leg:
    ct_age_cool = cooler.Cooler(f"/data/female-amb/loop/{group}/{group}/{group}.Q.cool")
    contacts = ct_age_cool.matrix(balance=False, as_pixels=True, join=True).fetch(f'{chrom}:{start}-{end}')
    tmp = contacts[(contacts['start1'] <= loop_start) & (contacts['end1'] >= loop_start)]
    tmp = tmp[(tmp['start2'] <= loop_end) & (tmp['end2'] >= loop_end)]
    age_count[group] = tmp


In [28]:
age_count['DG_Glut.8wk']

,chrom1,start1,end1,chrom2,start2,end2,count
17,chr3,152790000,152800000,chr3,152960000,152970000,0.006117
18,chr3,152790000,152800000,chr3,152970000,152980000,0.005098
35,chr3,152800000,152810000,chr3,152960000,152970000,0.006032
36,chr3,152800000,152810000,chr3,152970000,152980000,0.005160


In [29]:
age_count['DG_Glut.18mo']

,chrom1,start1,end1,chrom2,start2,end2,count
17,chr3,152790000,152800000,chr3,152960000,152970000,0.006136
18,chr3,152790000,152800000,chr3,152970000,152980000,0.005837
35,chr3,152800000,152810000,chr3,152960000,152970000,0.005541
36,chr3,152800000,152810000,chr3,152970000,152980000,0.005285
